In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Function for computing the average range of a dataset

In [2]:
def MRbar(data, L = 3):
    mrbar = 0
    # iterate for x_i, x_i+1 for i in data
    for (first, second) in list(zip(data[:-1],data[1:])):
        mrbar += abs(second-first)
    mrbar = mrbar/(len(data)-1)
    return mrbar

Setup up a function for Tabular CUSUM

In [3]:
def Cplus(xi, lastCplus, mean = 10, std = 1, k = .5):
    return max(0, xi - (mean + k*std) + lastCplus)
def Cneg(xi, lastCneg, mean = 10, std = 1, k = .5):
    return max(0, (mean - k*std) - xi + lastCneg)

Now, let's setup functions to run ARL simulations

In [4]:
def IXARLSim(mean0 = 10, L = 3, reps = 1000, mean1 = 10, std = 1):
    mrbar = MRbar(np.random.RandomState(1).normal(mean0, std, 10000),L) #need an estimate for MRbar
    runlengths = []
    for i in list(range(reps)):
        length = 0
        stop = False
        while not stop:
            value = np.random.RandomState(1000*i+length).normal(mean1, std)
            if value >= mean0 + (L*mrbar)/1.128 or value <= mean0 - (L*mrbar)/1.128:
                length += 1
                runlengths.append(length)
                stop = True
            else:
                length += 1
    return np.array(runlengths)


def CUSUMARLSim(reps = 1000, mean0 = 10, std = 1, mean1 = 10, k = .5, h = 5):
    runlengths = []
    for i in list(range(reps)):
        length = 0
        stop = False
        lastCplus = 0
        lastCneg = 0
        while not stop:
            value = np.random.RandomState(1000*i+length).normal(mean1, std)
            lastCplus = Cplus(value, lastCplus, mean0, std, k)
            lastCneg = Cneg(value, lastCneg, mean0, std, k)
            if lastCplus >= h*std or abs(lastCneg) >= h*std:
                length += 1
                runlengths.append(length)
                stop = True
            else:
                length += 1
    return np.array(runlengths)

def IXMRARLSim(L = 3, mean0 = 10, reps = 1000, mean1 = 10, std = 1, D = 4):
    mrbar = MRbar(np.random.RandomState(1).normal(mean0, std, 10000))
    runlengths = []
    for i in list(range(reps)):
        length = 0
        stop = False
        prevalue = mean0
        while not stop:
            value = np.random.RandomState(10000*i+length).normal(mean1, std)
            if value >= mean0 + (L*mrbar)/1.128 or value <= mean0 - (L*mrbar)/1.128:
                length += 1
                runlengths.append(length)
                stop = True
            if abs(value-prevalue) >= D*mrbar:
                length += 1
                runlengths.append(length)
                stop = True
            else:
                prevalue = value
                length += 1
    return np.array(runlengths)

## 1. Validate your code for IX and CUSUM chart for mean shifts. Pick a few ARL values from the OneNote page Comparison of Control Charts and make sure your numbers match the those in that page. Report your computational results.

|       Shift   Magnitude  |          IX   (L=3.5)  |          CUSUM    (h=5, k=1/2)  |          Combined   Chart        |
|--------------------------|------------------------|---------------------------------|----------------------------------|
|       0                  |          2149          |          465                    |          391                     |
|       0.25               |          281.2         |          139                    |          130.9                   |
|       0.5                |          155.2         |          38                     |          37.2                    |
|       1                  |          43.9          |          10.4                   |          10.2                    |
|       2                  |          6.2           |          4.0                    |          3.8                     |
|       3                  |          2.0           |          2.6                    |          2.1                     |
|       4                  |          1.2           |          2.0                    |          1.34                    |

I'll compare shift magnitudes of 0 and 3 for both IX and CUSUM

Starting with IX:

In [5]:
runlengths = IXARLSim(10,3.5, reps = 1000, mean1 = 10, std = 1)
print("Table's IX ARL for a mean shift of 0 : 2149")
print("My ARL: ",runlengths.mean())

Table's IX ARL for a mean shift of 0 : 2149
My ARL:  2362.656


In [6]:
runlengths = IXARLSim(10, 3.5, reps = 1000, mean1 = 13, std = 1)
print("Table's IX ARL for a mean shift of 3 : 2")
print("My ARL: ",runlengths.mean())

Table's IX ARL for a mean shift of 3 : 2
My ARL:  3.435


In [7]:
runlengths = CUSUMARLSim(reps = 1000, mean0 = 10, std = 1, mean1 = 10, k = .5, h = 5)
print("Table's CUSUM ARL for a mean shift of 0 : 465")
print("My ARL: ",runlengths.mean())

Table's CUSUM ARL for a mean shift of 0 : 465
My ARL:  492.667


In [8]:
runlengths = CUSUMARLSim(reps = 1000, mean0 = 10, std = 1, mean1 = 13, k = .5, h = 5)
print("Table's CUSUM ARL for a mean shift of 3 : 2")
print("My ARL: ",runlengths.mean())

Table's CUSUM ARL for a mean shift of 3 : 2
My ARL:  2.586


My computational values are all pretty close to the ones in the table,

## 2. Run IX and MR simultaneously. Generate ARL0 and ARL1 (0.5, 1, 2, 3 sigma shifts).

This will run two control charts at once. The same IX from before will be run:

<img src="Capture1.png">

with $d_2$ = 1.128

An MR chart will also be run:

<img src="Capture2.png">

with D = 4

In [9]:
meanshifts = [0,.5,1,2,3]
runlengthsIXMR = {}

for mshift in meanshifts:
    runlengthsIXMR[mshift] = IXMRARLSim(L = 3, mean0 = 10, reps = 1000, mean1 = 10+mshift, std = 1, D = 4)
    runlengthsIXMR[mshift] = np.array(runlengthsIXMR[mshift]).mean()

In [10]:
runlengthsIXMR = pd.DataFrame(runlengthsIXMR.values(), runlengthsIXMR.keys())

In [11]:
runlengthsIXMR=runlengthsIXMR.reset_index().rename(columns = {'index': 'Mean Shift', 0: 'ARL'})
runlengthsIXMR

,Mean Shift,ARL
0,0.0,305.381084
1,0.5,141.299242
2,1.0,45.278101
3,2.0,6.239882
4,3.0,1.994324


## 3. Simulate ARL0 and ARL1(0.5, 1, 2, 3 sigma shifts) for CUSUM chart for mean shifts. Make sure you choose a design ((k, h) for CUSUM so that its ARL0=370. Use Siegmund's (1985) equation to validate your results.

Siegmund's equations are:

<img src = Capture3.png width = 500px/>

In [12]:
from math import exp

h = 4.26609
k = .5
b = h + 1.666
delta = 0
Deltap = delta - k
Deltan = -delta - k

ARLp = (exp(-2*Deltap*b)+2*Deltap*b-1)/(2*Deltap**2)
ARLn = (exp(-2*Deltan*b)+2*Deltan*b-1)/(2*Deltan**2)
ARL = (1/ARLp + 1/ARLn)**-1
print("With (k,h) = ",(k,h), '\nARL = ', ARL)

With (k,h) =  (0.5, 4.26609) 
ARL =  370.00940887153797


In [13]:
meanshifts = [0,.5,1,2,3]
runlengthsCUSUM = {}

for mshift in meanshifts:
    runlengthsCUSUM[mshift] = CUSUMARLSim(reps = 1000, mean0 = 10, std = 1, mean1 = 10+mshift, k = k, h = h)
    runlengthsCUSUM[mshift] = np.array(runlengthsCUSUM[mshift]).mean()
runlengthsCUSUM = pd.DataFrame(runlengthsCUSUM.values(), runlengthsCUSUM.keys())
runlengthsCUSUM = runlengthsCUSUM.reset_index().rename(columns = {'index': 'Mean Shift', 0: 'ARL'})
runlengthsCUSUM

,Mean Shift,ARL
0,0.0,209.904
1,0.5,28.852
2,1.0,9.083
3,2.0,3.539
4,3.0,2.304


Not sure why the mean shift 0 is so low. Double-checked my code and everything seems to be running right. Additionally, I tested other values for the delta (delta = .5, 1, 2, etc.) in Siegmund's equations and those values were pretty close.

## 4. Implement Hawkins' (1981) CUSUM chart for variance change monitoring (n=1) <s>and run it simultaneously with the control chart you chose in Q3</s>. Generate ARL0 and ARL1(0.5, 1, 2, 3 sigma shifts)

The variance monitoring equation is:

<img src = Capture4.png/>

The writing is a little hard to read, but I believe it's supposed to update as:

$C^{+}_{i} = max[0,v_{i}-k+C^{+}_{i-1}]$

and that it is determined to be out of control when $C^{+}_{i}>h*\sigma$

In [14]:
from math import sqrt
def CUSUM_var_ARLSim(reps = 1000, mean0 = 10, std = 1, mean1 = 10, k = .5, h = 5):
    runlengths = []
    for i in list(range(reps)):
        length = 0
        stop = False
        y = 0
        Cv = 0
        while not stop:
            value = np.random.RandomState(1000*i+length).normal(mean1, std)
            y = (value-mean0)/std
            v = (sqrt(abs(y))-.822)/.349
            Cv = max(0, v-k+Cv)
            if Cv >= h*std:
                length += 1
                runlengths.append(length)
                stop = True
            else:
                length += 1
    return np.array(runlengths)

In [15]:
meanshifts = [0,.5,1,2,3]
runlengthsCUSUMv = {}

for mshift in meanshifts:
    runlengthsCUSUMv[mshift] = CUSUM_var_ARLSim(reps = 1000, mean0 = 10, std = 1, mean1 = 10+mshift, k = k, h = h)
    runlengthsCUSUMv[mshift] = np.array(runlengthsCUSUMv[mshift]).mean()
runlengthsCUSUMv = pd.DataFrame(runlengthsCUSUMv.values(), runlengthsCUSUMv.keys())
runlengthsCUSUMv = runlengthsCUSUMv.reset_index().rename(columns = {'index': 'Mean Shift', 0: 'ARL'})
runlengthsCUSUMv

,Mean Shift,ARL
0,0.0,446.309
1,0.5,126.165
2,1.0,22.533
3,2.0,4.667
4,3.0,2.676


## 5. Combine the results from Q3 (CUSUM-M) and Q4 (CUSUM-V) to compute the ARL0 and ARL1(0.5, 1, 2, 3 sigma shifts) (i.e. 1/ARL=1/ARL-M+1/ARL-V) to compute the approximate ARL values when both CUSUM for mean and CUSUM for variance are running simultaneously.  For those who use Python for simulation, run both CUSUM-M and CUSUM-V simultaneously and tabulate the ARL values. Do these values match with those generated from the 1/ARL equation?

In [16]:
def CUSUM_var_mean_ARLSim(reps = 1000, mean0 = 10, std = 1, mean1 = 10, k = .5, h = 5):
    runlengths = []
    for i in list(range(reps)):
        length = 0
        stop = False
        lastCplus = 0
        lastCneg = 0
        y = 0
        Cv = 0
        while not stop:
            value = np.random.RandomState(1000*i+length).normal(mean1, std)
            lastCplus = Cplus(value, lastCplus, mean0, std, k)
            lastCneg = Cneg(value, lastCneg, mean0, std, k)
            y = (value-mean0)/std
            v = (sqrt(abs(y))-.822)/.349
            Cv = max(0, v-k+Cv)
            if lastCplus >= h*std or abs(lastCneg) >= h*std or Cv >= h*std:
                length += 1
                runlengths.append(length)
                stop = True
            else:
                length += 1
    return np.array(runlengths)

In [17]:
meanshifts = [0,.5,1,2,3]
runlengthsCUSUMmv = {}

for mshift in meanshifts:
    runlengthsCUSUMmv[mshift] = CUSUM_var_mean_ARLSim(reps = 1000, mean0 = 10, std = 1, mean1 = 10+mshift, k = k, h = h)
    runlengthsCUSUMmv[mshift] = np.array(runlengthsCUSUMmv[mshift]).mean()
runlengthsCUSUMmv = pd.DataFrame(runlengthsCUSUMmv.values(), runlengthsCUSUMmv.keys())
runlengthsCUSUMmv = runlengthsCUSUMmv.reset_index().rename(columns = {'index': 'Mean Shift', 0: 'ARL'})
runlengthsCUSUMmv

,Mean Shift,ARL
0,0.0,166.031
1,0.5,28.082
2,1.0,9.058
3,2.0,3.536
4,3.0,2.304


I didn't see an equation for finding the ARL for of the CUSUM variance control chart, so I'll use the ARL computed in the table above instead along with the mean ARL from Siegmund's equations.

In [18]:
ARL = (1/446.309+1/370)**-1
print(ARL)

202.29389851147053


If I'm able to use the same equations, then I assume that it would be:

In [19]:
h = 4.26609
k = .5
b = h + 1.666
delta = 0
Deltap = delta - k
Deltan = -delta - k
Deltav = delta-k

ARLp = (exp(-2*Deltap*b)+2*Deltap*b-1)/(2*Deltap**2)
ARLn = (exp(-2*Deltan*b)+2*Deltan*b-1)/(2*Deltan**2)
ARLv = (exp(-2*Deltav*b)+2*Deltav*b-1)/(2*Deltav**2)
ARL = (1/ARLp + 1/ARLn + 1/ARLv)**-1
print("With (k,h) = ",(k,h), '\nARL = ', ARL)

With (k,h) =  (0.5, 4.26609) 
ARL =  246.672939247692


## Tabulate all ARL0 and ARL1 values in rows and different methods in columns.   Compare the results from Q2 (IX and MR) and Q5 (CUSUM-M and CUSUM-V). Which pair of charts would you recommend? Why?

In [20]:
comb = pd.DataFrame([0,.5,1,2,3], columns = ['Mean Shift'])
comb['IX MR'] = runlengthsIXMR['ARL']
comb['CUSUM Mean'] = runlengthsCUSUM['ARL']
comb['CUSUM Var'] = runlengthsCUSUMv['ARL']
comb['CUSUM Combined'] = runlengthsCUSUMmv['ARL']

In [21]:
comb

,Mean Shift,IX MR,CUSUM Mean,CUSUM Var,CUSUM Combined
0,0.0,305.381084,209.904,446.309,166.031
1,0.5,141.299242,28.852,126.165,28.082
2,1.0,45.278101,9.083,22.533,9.058
3,2.0,6.239882,3.539,4.667,3.536
4,3.0,1.994324,2.304,2.676,2.304


I don't really see an obvious choice between IX MR and the combined CUSUM charts. That choice would probably just be an application preference. If you want to react to mean shift faster, then you should choose the combined CUSUM chart. If you want to have fewer false alarms, then you should choose the IX MR chart. Overall, I'd probably choose the combined CUSUM chart as waiting 141 samples to detect a mean shift of 0.5 seems a little long to me,